In [10]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as pltcolors

import torch as t
import torch_geometric as ptgeo

from torch_geometric.utils import to_networkx, from_networkx
import networkx as nx

from gninvert.functions import run_GN, gn_time_series, make_color_scale,\
gdisplay, run_and_draw, generate_training_data,\
generate_graphs_from_connections, generate_grid_edge_index
from gninvert.gns import MultiDiffusionGN, EquationGN, ActivatorInhibitorGN
from gninvert.gnns import LinearGNN
from gninvert.graph_compare import graph_compare, model_compare
import gninvert

import ipywidgets as widgets

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [32]:
plt.rcParams['figure.figsize'] = [10, 4] # set plot size below

In [87]:
gn = ActivatorInhibitorGN(0.16, 0.10, 0.3)
gdata = generate_graphs_from_connections(
    generate_grid_edge_index(6),
    node_feature_num=3,
    num=1
)[0]

# make the cell sizes homogenous
gdata.x[:, 0] = 0.5

# activator/inhibitor all start in one corner
gdata.x[:, 1] = 0
gdata.x[:, 2] = 0
gdata.x[0, 1] = 4
gdata.x[1, 1] = 1
gdata.x[0, 2] = 4
gdata.x[1, 2] = 1

time_series = gn_time_series(gn, 300, gdata)

In [88]:
def f(x):
    gdisplay(
        time_series[x],
        color_scales = [
            make_color_scale(0, 1, plt.cm.Greys),
            make_color_scale(0, 1, plt.cm.BuGn),
            make_color_scale(0, 1, plt.cm.Reds)
        ]
    )
    print(x)

widgets.interact(
    f,
    x=widgets.IntSlider(min=0, max=len(time_series)-1, step=1, value=0)
)

interactive(children=(IntSlider(value=0, description='x', max=300), Output()), _dom_classes=('widget-interact'…

<function __main__.f(x)>